In [39]:
import numpy as np
import pandas as pd 
import pickle
import os
import pandas as pd
import numpy as np
import math
import datetime
import sys
import scipy.stats as stats
# from sklearn.pipeline import Pipeline
# from sklearn import linear_model, decomposition
# from sklearn.svm import LinearSVC
# from sklearn.svm import SVC
# from sklearn.svm import SVR
# from sklearn.metrics import precision_recall_curve
# from sklearn.linear_model import SGDClassifier, LogisticRegression
# from sklearn.metrics import classification_report
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn import decomposition
# from sklearn.linear_model import LogisticRegression
# from sklearn import svm
# from sklearn.grid_search import GridSearchCV
# from sklearn.externals import joblib
# Added version check for recent scikit-learn 0.18 checksok ca
from distutils.version import LooseVersion as Version
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
from collections import OrderedDict
from sklearn.svm import SVC # "Support Vector Classifier"
# from sklearn.decomposition import FastICA
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_regression
# from sklearn.feature_selection import SelectPercentile, f_classif
# from sklearn.cross_validation import KFold
# from sklearn.cross_validation import cross_val_score
# from sklearn import preprocessing

# from mpl_toolkits.mplot3d import Axes3D
from math import sqrt

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

#### ticker ####
ticker = 'test_SYNT_2states' #testing a new synthetic ticker
# paths
data_dir = os.getenv('FINANCE_DATA') #main directory
features_path = '/home/ak/Data/features_models/features/' #where features are saved
labels_path = '/home/ak/Data/features_models/labels' #where labels are saved
ticker_labels_path = os.path.join(labels_path, ticker + '/NON_DIRECTIONAL')
main_path = '/home/ak/Data/features_models/'
models_path = os.path.join(main_path, 'models')
ticker_models_path = os.path.join(models_path, ticker)

#### predictions path ####
###functions##'

def fwd_dates(_dates_list, _key_date):
    # returns a list of dates that are forward from the key_date
    fwd_dates_list = [i for i in _dates_list if i > _key_date]
    return fwd_dates_list

list_models =os.listdir(ticker_models_path)
data_dates= [os.path.splitext(item)[0] for item in os.listdir(os.path.join(data_dir, ticker))]
model_dates=[list_models[item].split('_')[2] for item, _ in enumerate(list_models)]
##
class DataLoader(object):
    def __init__(self, path_, ticker):
        self.main_path = path_
        self.ticker = ticker
        self.labels_path = os.path.join(self.main_path, 'labels')
        self.features_path = os.path.join(self.main_path, 'features')
        self.ticker_labels_path = os.path.join(self.labels_path, self.ticker)
        self.ticker_features_path = os.path.join(self.features_path, self.ticker)

    def ticker_features(self, date):
        file_loc = os.path.join(self.ticker_features_path, str(date) + '.pickle')
        with open(file_loc, 'rb') as handle:
            ticker_features = pickle.load(handle)
        return ticker_features

    def ticker_labels_pickle(self, date):
        file_loc = os.path.join(self.ticker_labels_path, str(date) + '.pickle')
        with open(file_loc, 'rb') as handle:
            ticker_labels = pickle.load(handle)
        return ticker_labels

    def ticker_labels_csv(self, date):
        file_loc = os.path.join(self.ticker_labels_path, str(date) + '.csv')
        ticker_labels = pd.read_csv(file_loc)
        return ticker_labels

    @staticmethod
    def open_pickle_file(path, pickle_file):
        file_loc = os.path.join(path, pickle_file)
        pickle_to_file = pickle.load(open(file_loc, "rb"))
        return pickle_to_file

    @staticmethod
    def get_date_from_file(file_, numb_):
        return os.path.splitext(file_[numb_])[0]


In [43]:
from hsmm_core.hmm import hmm_engine
from hsmm_core.observation_models import ExpIndMixDiracGauss
from hsmm_core.feature_spaces import hmm_features
from hsmm_core.hmm import hmm_calibration
from hsmm_core.data_utils import load_data, TradingHours
from hsmm_core.labelling import DataLabellingSimple
from hsmm_core.consts import ThresholdMethod, LabellingChoice
import pickle
from hsmm_core.consts import InitialisationMethod


In [37]:
list_models[1]
filename = os.path.join(ticker_models_path, list_models[1])
loaded_model = pickle.load(open(filename, 'rb'))['SVC'].best_estimator_
#loaded_model.best_estimator_

In [45]:

data_cls = DataLoader(path_=main_path, ticker=ticker)

In [47]:
#  Parameters -setting up the HMM etc
no_states = 2
sigmas = [0.05, 0.002]  # fast and slow
# Duration is measured in seconds for now (to be revised). lambda units are seconds^{-1}
# so here we consider

lambdas = [1. / 35., 1. / 10.]
weights = [0.1, 0.6]

obs_model = ExpIndMixDiracGauss(no_states)
obs_model.set_up_initials(priors={'sigmas': sigmas, 'lambdas': lambdas, 'weights': weights})

hmm_ = hmm_engine(obs_model, no_states)

# set up some priors
tpm = np.array([[0.4, 0.6], [0.7, 0.3]])
pi = np.array([0.4, 0.6])
hmm_.set_up_initials(priors={'tpm': tpm, 'pi': pi})
rng = np.random.RandomState(1234)
trd_hours_filter = TradingHours.all_trading_day



In [27]:
#load the model
#get fwd dates
#get the feature and label data
#do prediction?
#produce output

In [48]:
init_params = {
    "obs_model_params": {
        'obs_model_name': 'ExpIndMixDiracGauss',
        'em_init_method': InitialisationMethod.cluster

    },
    "hidden_model_params": {
        'no_hidden_states': no_states,
        'pi': pi,
        'tpm': tpm,
        'em_init_method': InitialisationMethod.uniform
    },
    "update_tag": 'tpsml'
}

data = load_data(ticker, which_trading_hours=TradingHours.all_trading_day) #prob dont need this

hmm_calibration_engine = hmm_calibration(no_parallel_procs=None,
                                             init_params=init_params)

trained_hmms = hmm_calibration_engine.hmm_fit_func(ticker, data, trd_hours_filter,
                                                       force_recalc=False)

Calibrating hmm on date 20170630
Number of points in data set is 5000, number of points with large price change 3315
Calibrating hmm on date 20170710
Number of points in data set is 5000, number of points with large price change 3404
Calibrating hmm on date 20170613
Number of points in data set is 5000, number of points with large price change 3345
Calibrating hmm on date 20170612
Number of points in data set is 5000, number of points with large price change 3355
Calibrating hmm on date 20170712
Number of points in data set is 5000, number of points with large price change 3384
Calibrating hmm on date 20170616
Number of points in data set is 5000, number of points with large price change 3375
Calibrating hmm on date 20170615
Number of points in data set is 5000, number of points with large price change 3332
Calibrating hmm on date 20170614
Number of points in data set is 5000, number of points with large price change 3359
Calibrating hmm on date 20170619
Number of points in data set is

In [52]:
model_dates[1]
hmm_fwd_dates = fwd_dates(_dates_list=data_dates, _key_date=model_dates[1])  # create fwd out of sample dates

In [69]:

#         features_load = feature_engine.generate_features(data[date])
#         labels_load = pd.read_csv(os.path.join(ticker_labels_path,str(date)+'.csv'))

In [62]:
for _, date_hmm in trained_hmms.iteritems():
    feature_engine = hmm_features(date_hmm) #date_hmm is every fitted_model_date
    fwd_features = feature_engine.generate_features(data_dic[fwd_date])
    fwd_labels = data_cls.ticker_labels_csv(date=fwd_date)
    features_fwd, labels_fwd = remove_nans(fwd_features, fwd_labels)
    features_load = feature_engine.generate_features(data[date])
    labels_load = pd.read_csv(os.path.join(ticker_labels_path,str(date)+'.csv'))
    features, labels_clean = remove_nans(features_load, labels_load)


In [66]:
for date, _ in trained_hmms.iteritems():
    feature_engine = hmm_features(date_hmm) #model engine for feature generation
    hmm_fwd_dates = fwd_dates(_dates_list=data_dates, _key_date=date)  # create fwd out of sample dates

20170621
20170630
20170613
20170612
20170712
20170616
20170710
20170614
20170619
20170606
20170627
20170615
20170703
20170705
20170704
20170707
20170706
20170626
20170601
20170602
20170622
20170623
20170620
20170607
20170608
20170609
20170711
20170628
20170629
20170605
